In [ ]:
import csv
import re
import urllib
import requests
from bs4 import BeautifulSoup
#site="https://www.zameen.com/Property/peshawar_awt_housing_scheme_badabair_house_is_available_for_sale-8920348-986-1.html"
#site="https://www.zameen.com/Property/islamabad_bahria_town_amazing_1_kanal_house_for_sale_in_bahria_town-4088817-383-1.html"
#site="https://www.zameen.com/search/results.html?rd_purpose=1&property_type=9&view_sale=0&sqft_conv_unit=-1&sqft_unit=Select%20Unit&sqft_custom=1&filter_city=17&page=1&tab_city=17&tab_type=9&tab_purpose=1&tab_sqft_unit=Select%20Unit&tab_sqft_conv_unit=-1&tab_sqft_custom=1&tab_search=1"

url1="https://www.zameen.com/search/results.html?rd_purpose=1&property_type=9&view_sale=0&sqft_conv_unit=-1&sqft_unit=Select%20Unit&sqft_custom=1&filter_city=17&page="
url2="&tab_city=17&tab_type=9&tab_purpose=1&tab_sqft_unit=Select%20Unit&tab_sqft_conv_unit=-1&tab_sqft_custom=1&tab_search=1"
for page_numbers in range(458,461):
    
    
    import winsound
    duration = 300 # millisecond
    freq = 440  # Hz
    winsound.Beep(freq, duration)

    
    
    
    urlpage=url1+str(page_numbers)+url2
    headers = {'User-Agent':'Mozilla/5.0'}
    page = requests.get(urlpage)
    data = page.content 
    soup = BeautifulSoup(data, "html.parser")
    soup.prettify()
    first = soup.find_all('div', {'class' : 'ls clr'})
    houses_links=[]
    for link in first:
        houses_links.append(link.a["href"])
    
    for per_page_add in range(0, len(houses_links)-1):
        headers = {'User-Agent':'Mozilla/5.0'}
        page = requests.get(houses_links[per_page_add])
        data = page.content
        soup = BeautifulSoup(data, "html.parser")
        soup.prettify()
        data_set=[0]*62
        
        house_title_raw = soup.find('h1', {'class' : 'l pv_h1'})
        house_title=house_title_raw.text
        data_set[0]=house_title

        house_address_raw = soup.find('h2', {'class' : 'clr lst_view_font_16'})
        house_address= house_address_raw.text
        data_set[1]=house_address
        ##print(house_address)
        address_spliting=(house_address).split()
        city=address_spliting[-1]
        #data_set[2]=city
        if len(address_spliting)>1:
            colony=address_spliting[-2]
            data_set[2]=colony
        
        homeaddress=''.join(map(str, address_spliting[0:-1]))
        homeaddresss = homeaddress.replace(",", " ")
        data_set[1]=homeaddresss
        
        data_set[3]=city
        
        ##print(city)
        font = soup.find_all('font', {'class' : 'bg_info'})
        first_lines = [span.get_text() for span in font] #print (lines[0]) #Price #print (lines[1]) #Area 
        price_str=first_lines[0]
        price_word=price_str.split() #print (price_word[1]) #print (price_word[2])
        if price_word[2]=='Crore':
            pw=[float(s) for s in re.findall(r'[-+]?\d*\.\d+|\d+',first_lines[0] )]
            pws=round((pw[0])*100000) # thousand scale 1k
            print(pws)
            data_set[4]=pws
        if price_word[2]=='Lakh':
            pw=[float(s) for s in re.findall(r'[-+]?\d*\.\d+|\d+',first_lines[0])]
            pws=round((pw[0])*1000) # thousand scale 1k
            print(pws)
            data_set[4]=pws
        area_str=first_lines[1]
        area_word=area_str.split() #print (area_word[0]) #print (area_word[1])
        if area_word[1]=='Kanal':
            aw=[float(s) for s in re.findall(r'[-+]?\d*\.\d+|\d+',first_lines[1] )]
            aws=round((aw[0])*20) # scale of marla
            #print(aws)
            data_set[5]=aws
        if area_word[1]=='Marla':
            aw=[float(s) for s in re.findall(r'[-+]?\d*\.\d+|\d+',first_lines[1])]
            aws=round((aw[0])*1) # scale of marla
            #print(aws)
            data_set[5]=aws
        first = soup.find_all('span', {'class' : 'l pv_ov_title'})
        spans = soup.find_all('span', {'class' : 'r pv_ov_value '})
        aa_tit = [span_tit.get_text() for span_tit in first]
        uu_val = [span_val.get_text() for span_val in spans]
        data_set[9]="nodatabedrooms"
        data_set[10]="nodatabathroms"
        data_set[12]="nodataownership"
        data_set[13]="nodataoccupancy"
        for num in range(0,len(first)):
            b_spl=aa_tit[num].split()
            if b_spl[0]=="Zameen":
                data_set[6]=uu_val[num]
            if b_spl[0]=="Type":
                data_set[7]=uu_val[num]
            if b_spl[0]=="Status":
                data_set[8]=uu_val[num]           
            if b_spl[0]=="Bedrooms":
                data_set[9]=uu_val[num]           
            if b_spl[0]=="Bathrooms":
                data_set[10]=uu_val[num]          
            if b_spl[0]=="Added:":
                #print("wegot1")
                data_set[11]=uu_val[num-2]
                #print(num)
            if b_spl[0]=="Ownership":
                data_set[12]=uu_val[num-2]
                #print("wegot2")
                #print(num)
            if b_spl[0]=="Occupancy":
                data_set[13]=uu_val[num-2]
                #print("wegot3")
                #print(num)
        
        feature_label = soup.find_all('label',attrs={"class":"l label"})
        typ = soup.find_all('div',attrs={'class':'bgw ros features_inner'})
        if len(typ)!=0:
            feature_exist=typ[0].find_all('span',{'class':'l'})
            #feature_exist
        
        p = 0
        for n in range(0, len(feature_label)-1):
            if feature_label[n].text=="Built In Year":
                data_set[14]=feature_exist[p].text
                #print("Built In Year:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Central Heating":
                data_set[15]="Central heating"
                #print("Central heating")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Double Glazed Windows":
                data_set[16]="Double Glazed Windows"
                #print("Double Glazed Windows")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Electricity Backup":
                data_set[17]=feature_exist[p].text
                #print("Electricity Backup:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Flooring":
                data_set[18]=feature_exist[p].text
                #print("Flooring:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Floors":
                data_set[19]=feature_exist[p].text
                #print("Floors:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Other Main Features":
                data_set[20]=feature_exist[p].text
                #print("Other Main Features:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="View":
                data_set[21]=feature_exist[p].text
                #print("View:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Central Air Conditioning":
                data_set[22]="Central Air Conditioning"
                #print("Central Air Conditioning")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Furnished":
                data_set[23]="Furnished"
                #print("Furnished")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Parking Spaces":
                data_set[24]=feature_exist[p].text
                #print("Parking Spaces:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Waste Disposal":
                data_set[25]="Waste Disposal"
                #print("Waste Disposal")
                #print(n)
                #print(p)
                p= p + 2

            if feature_label[n].text=="Bathrooms":
                data_set[26]=feature_exist[p].text
                #print("Bathrooms:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Bedrooms":
                data_set[27]=feature_exist[p].text
                #print("Bedrooms:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Dining Room":
                data_set[28]="Dining Room"
                #print("Dining Room")
                #print(n)
                #print(p)        
                p= p + 2
            if feature_label[n].text=="Drawing Room":
                data_set[29]="Drawing Room"
                #print("Drawing Room")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Kitchens":
                data_set[30]=feature_exist[p].text
                #print("Kitchens:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Laundry Room":
                data_set[31]="Laundry Room"
                #print("Laundry Room")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Lounge Or Sitting Room":
                data_set[32]="Lounge Or Sitting Room"
                #print("Lounge Or Sitting Room")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Other Rooms":
                data_set[33]=feature_exist[p].text
                #print("Other Rooms:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Powder Room ":
                data_set[34]="Powder Room"
                #print("Powder Room")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Prayer Room":
                data_set[35]="Prayer Room"
                #print("Prayer Room")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Servant Quarters":
                data_set[36]=feature_exist[p].text
                #print("Servant Quarters:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Steam Room":
                data_set[37]="Steam Room"
                #print("Steam Room")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Store Rooms":
                data_set[38]=feature_exist[p].text
                #print("Store Rooms:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1
            if feature_label[n].text=="Study Room":
                data_set[39]="Study Room"
                #print("Study Room")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Gym":
                data_set[40]="Gym"
                #print("Gym")
                #print(n)
                #print(p)
                p= p + 2
    
            if feature_label[n].text=="Broadband Internet Access":
                data_set[41]="Broadband Internet Access"
                #print("Broadband Internet Access")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Intercom":
                data_set[42]="Intercom"
                #print("Intercom")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Satellite Or Cable TV Ready":
                data_set[43]="Satellite Or Cable TV Ready"
                #print("Satellite or Cable TV Ready")
                #print(n)
                #print(p)
                p= p + 2
            if feature_label[n].text=="Other Business And Communication Facilities":
                data_set[44]=feature_exist[p].text
                #print("Other Business and Communication Facilities:"+feature_exist[p].text)
                #print(n)
                #print(p)
                p= p + 1

            if feature_label[n].text=="Lawn Or Garden":
                data_set[45]="Lawn Or Garden"
                ##print("Lawn Or Garden")
                ##print(n)
                ##print(p)
                p= p + 2
            if feature_label[n].text=="Other Healthcare And Recreation Facilities":
                data_set[46]=feature_exist[p].text
                ##print("Other Healthcare And Recreation Facilities:"+feature_exist[p].text)
                p= p + 1
            if feature_label[n].text=="Jacuzzi":
                data_set[47]="Jacuzzi"
                ##print("Jacuzzi")
                p= p + 2
            if feature_label[n].text=="Swimming Pool":
                data_set[48]="Swimming Pool"
                ##print("Swimming Pool")
                p= p + 2
            if feature_label[n].text=="Sauna":
                data_set[49]="Sauna"
                ##print("Sauna")
                p= p + 2

            if feature_label[n].text=="Distance From Airport (kms)":
                data_set[50]=feature_exist[p].text
                ##print(feature_exist[p].text)
                p= p + 1
            if feature_label[n].text=="Nearby Hospitals":
                data_set[51]=feature_exist[p].text
                ##print(feature_exist[p].text)
                p= p + 1
            if feature_label[n].text=="Nearby Public Transport Service":
                data_set[52]=feature_exist[p].text
                ##print(feature_exist[p].text)
                p= p + 1
            if feature_label[n].text=="Nearby Restaurants":
                data_set[53]=feature_exist[p].text
                ##print(feature_exist[p].text)
                p= p + 1
            if feature_label[n].text=="Nearby Schools":
                data_set[54]=feature_exist[p].text
                ##print(feature_exist[p].text)
                p= p + 1
            if feature_label[n].text=="Nearby Shopping Malls":
                data_set[55]=feature_exist[p].text
                ##print(feature_exist[p].text)
                p= p + 1
            if feature_label[n].text=="Other Nearby Places":
                data_set[56]=feature_exist[p].text
                ##print(feature_exist[p].text)
                p= p + 1

            if feature_label[n].text=="Facilities For Disabled":
                data_set[57]="Facilities For Disabled"
                ##print("Facilities For Disabled")
                p= p + 2
            if feature_label[n].text=="Maintenance Staff":
                data_set[58]="Maintenance Staff"
                ##print("Maintenance Staff")
                p= p + 2
            if feature_label[n].text=="Other Facilities":
                data_set[59]=feature_exist[p].text
                ##print(feature_exist[p].text)
                p= p + 1
            if feature_label[n].text=="Security Staff":
                data_set[60]="Security Staff"
                ##print("Security Staff")
                p= p + 2
        
        data_set[61]=page_numbers
        afaq=[data_set]
        csvfile = open('output.csv', 'a')
        for row in afaq:
            #line = ','.join(row)
            values = ','.join(str(v) for v in row)
            csvfile.write(values + '\n')



In [4]:
page_numbers

460

In [ ]:
data_set
#homeaddress = ['aaa', 'fff','fffs',]
#homeaddress=' '.join(homeaddress[0:-1])
#homeaddress
feature_exist


In [ ]:
my_lst_str = ''.join(map(str, homeaddress))
print(my_lst_str)

In [ ]:
address_spliting

In [ ]:
afaq=[data_set]
csvfile = open('afaq.csv', 'a')
for row in afaq:
    #line = ','.join(row)
    values = ','.join(str(v) for v in row) 
    csvfile.write(values + '\n')

In [ ]:
var string=" , this,  is a ,,, test string , , to find regex,,in js.  , ";
res = str.replace(/[, ]+/g, " ").trim();

In [ ]:
homeaddress.replace(",", " ")

In [ ]:
page_numbers